# Gmail Newsletter Exploration

### Goal
- explore the features of the Gmail api 
- gather relevant newsletter mails form the mailbox
- scrape newsletter based on thier Design so that i can standardize the database article mentioned in the mails

### DB-Layout
- id
- url
- content name
- type of content
- full text or snippet of content

In [2]:
# imports
from google.oauth2.credentials import Credentials   # handel auth
from googleapiclient.discovery import build         # used to make the queries

import os.path                  # handle paths
import base64                   # decrypt mail data
from bs4 import BeautifulSoup   # parse data after decryption

relevant paramters for list()
- maxResult: default 100 / up to 500
- q: only returns messages matching a query "from:lon@dataelixir.com"     
- pageToken: "retrive a specific page of result in a list"

In [31]:
# Define SCOPES
# relevant for auth
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

pathToToken = '/Users/markusmuller/python/projects/gmail-newsletter-db/token.json'

creds = Credentials.from_authorized_user_file(pathToToken, SCOPES)

# connect to Gmail api
service = build('gmail', 'v1', credentials=creds)

# request a list of all the messages
result = service.users().messages().list(userId='markus.mueller.ds@gmail.com').execute()

In [35]:
print(type(result))
# frirst 10 result of 
print(result['messages'][:10])
print('Next page Token: ',  result['nextPageToken'])
print('length: ', len(result['messages']))

<class 'dict'>
[{'id': '17bed324d52f36ee', 'threadId': '17bed324d52f36ee'}, {'id': '17beb2ea7a06eb5e', 'threadId': '17beb2ea7a06eb5e'}, {'id': '17be8bcdaa338b35', 'threadId': '17be8bcdaa338b35'}, {'id': '17be5f07abd13e84', 'threadId': '17be5f07abd13e84'}, {'id': '17be5ac70eaf3f1d', 'threadId': '17be5ac70eaf3f1d'}, {'id': '17be368851c51489', 'threadId': '17be368851c51489'}, {'id': '17be067784b98401', 'threadId': '17be067784b98401'}, {'id': '17bdf62d721058f5', 'threadId': '17bdf62d721058f5'}, {'id': '17bdafc60ea3954c', 'threadId': '17bdafc60ea3954c'}, {'id': '17bd51b3bf685077', 'threadId': '17bd51b3bf685077'}]
Next page Token:  10546748924264687126
length:  100


Now we can use the ids to get the content of the mails

In [19]:
msg_ids = result.get('messages')

In [28]:
# get the data for one mail
id_ = '17bed324d52f36ee'
txt = service.users().messages().get(userId='me', id=id_).execute()


message object https://developers.google.com/gmail/api/reference/rest/v1/users.messages#Message:
- id: ID of the message
- threadId: The ID of the thread the message belongs to. 
- labelIds: List of IDs of labels applied to this message.
- snippet: A short part of the message text.
- historyId: The ID of the last history record that modified this message.
- internalDate: The internal message creation timestamp (epoch ms)
- payload: The parsed email structure in the message parts. (here is the relevant data)

In [29]:
print(txt.keys())

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])


In [ ]:
payload = txt['payload']

Next cell saves the output in an external file os I can view the content 

In [42]:
%%capture cap --no-stderr
print(payload)
with open('test.txt', 'w') as f:
    f.write(str(cap))

In [52]:
payload.keys()

dict_keys(['partId', 'mimeType', 'filename', 'headers', 'body', 'parts'])

In [53]:
print(payload['headers'])

[{'name': 'Delivered-To', 'value': 'markus.mueller.ds@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6e04:139c:0:0:0:0 with SMTP id be28csp2135701imb;        Wed, 15 Sep 2021 23:01:42 -0700 (PDT)'}, {'name': 'X-Google-Smtp-Source', 'value': 'ABdhPJyxdMCapZIZ5Y2skh15iwb4zjdvYJLLsM4QvWJA0SJ0kRLSxC91nVNXVGAVCOPtoTAajtQ2'}, {'name': 'X-Received', 'value': 'by 2002:a05:6902:110d:: with SMTP id o13mr4912149ybu.530.1631772101846;        Wed, 15 Sep 2021 23:01:41 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1631772101; cv=none;        d=google.com; s=arc-20160816;        b=BQI6VxIPo295XbyRQYuNmSHaNs3p/jg82bkgLzhoIwVuSh9IfN+IQdEmXONgWqVhbJ         A8gtVP29JeiD2nEpEd+j/jQoiHlPowC2/1v+JIn3XcVIWKvu/Yzade4fEnGRjNOiFfMC         QNjRNBYEmW0GTwEMtEFYRZWjKf0btWjR+igqfKaPhv/4CekxQU7IdnC/mewIJeTYDsX1         h0SWLsDTwqVy5XHJUy8xszprwtvJCWqSmfpLK7r+EL1V3PuBzc0EzUunYoyjTOA3I9Tk         9t68bAN+Gc3Vo9IWsYLod+gJj+kpM550alSh+QQgA5CqbtbN9pilltmlE4+tyJkES/WO         qKzg=='}, {'na

In [22]:
for msg in msg_ids:
    print(msg['id'])

17bed324d52f36ee
17beb2ea7a06eb5e
17be8bcdaa338b35
17be5f07abd13e84
17be5ac70eaf3f1d
17be368851c51489
17be067784b98401
17bdf62d721058f5
17bdafc60ea3954c
17bd51b3bf685077
17bd4d4f6a8e5bd7
17bd41ed7e3689c9
17bd157766588699
17bd053251a031b3
17bcf5e615f87ecd
17bcef38c960fc3b
17bca0bc853a32f3
17bc92883487c59a
17bc6bf1817afc13
17bc4a99883075e1
17bc47ca455dea43
17bc1e3e4c595966
17bc17764ecf3708
17bbf853dfebce27
17bba2e6ee21ba8a
17bb7432c5ae73c8
17bb6870bb68b786
17bb0770b57f10ee
17bad571fe5322f8
17bab51d2e5a93ff
17bab5189de5ff8a
17bab32fc35f68e2
17ba5c0f63e5d7a6
17ba598e729bf32b
17ba349206733d4b
17ba27ef25924242
17ba0a53c538f36a
17b9de70877d3d32
17b9dd5d1c159068
17b9d97f81a5b755
17b9d3e197b14d4f
17b8c4228e41efa3
17b8c0b2c93af7d1
17b890045309533f
17b880171729f007
17b87457879bb57f
17b86ab90b36ee35
17b829a3d641cc67
17b7f2b2004c9889
17b7c8c29c863548
17b7c536cd4056ff
17b79c95aed7e0a6
17b797edd35042ff
17b78787b99689b8
17b77a030916434b
17b7213fcd2d544f
17b6f44da0eae264
17b68bd606a5edde
17b68035bfd5b6